# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#load and read csv file
city_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
city_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Cloudiness,Humidity,Wind Speed,Country,Date
0,0,rikitea,-23.1203,-134.9692,76.59,100.0,86.0,16.22,PF,1.623046e+09
1,1,carnarvon,-24.8667,113.6333,78.60,100.0,48.0,11.12,AU,1.623046e+09
2,2,marcona,5.4112,100.3354,89.89,40.0,70.0,3.51,MY,1.623046e+09
3,3,georgetown,60.0158,41.0614,65.35,65.0,68.0,9.75,RU,1.623046e+09
4,4,syamzha,-53.1500,-70.9167,37.51,0.0,87.0,9.40,CL,1.623046e+09


In [64]:
city_data.count()

Unnamed: 0    560
City          560
Lat           560
Lng           560
Max Temp      560
Cloudiness    560
Humidity      560
Wind Speed    560
Country       557
Date          560
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [65]:
#configure maps
gmaps.configure(api_key=g_key)

In [75]:
#Use the Lat and Lng as location
locations = city_data[["Lat", "Lng"]]

#Humidity as the weight
humidity = city_data["Humidity"]
weight = city_data["Humidity"]

In [76]:
# Plot Heatmap
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use the gmaps.figure passing a zoom_level of 2 and a center point so the map displays properly at a readable size
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

## Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [68]:
# find ideal cities that fit weather conditions
ideal_city = city_data.loc[(city_data['Max Temp'] < 85) & (city_data['Max Temp']>70)\
                         & (city_data['Wind Speed'] < 10)\
                         & (city_data['Cloudiness'] == 0)] 
#store as a new variable
ideal_city = ideal_city.copy()

#clean up DataSet by remooving NAN
clean_ideal = ideal_city.dropna()

clean_ideal

,Unnamed: 0,City,Lat,Lng,Max Temp,Cloudiness,Humidity,Wind Speed,Country,Date
177,177,raudeberg,-20.7333,139.5000,80.37,0.0,22.0,8.05,AU,1.623046e+09
292,292,vad,18.2164,42.5053,80.85,0.0,18.0,3.98,SA,1.623046e+09
302,302,popondetta,47.8667,88.1167,71.44,0.0,22.0,9.26,CN,1.623046e+09
321,321,irbil,52.4994,73.0975,74.88,0.0,25.0,4.68,KZ,1.623046e+09
394,394,pitrufquen,52.3710,84.3873,72.88,0.0,48.0,9.93,RU,1.623046e+09
423,423,axim,31.6100,34.7642,76.78,0.0,27.0,2.26,IL,1.623046e+09
436,436,pierre,36.1683,54.3480,83.73,0.0,25.0,2.51,IR,1.623046e+09


# # Hotel Map¶

* Set parameters to search for hotels with 5000 meters.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [77]:
#Store into variable named hotel_df
hotel_df = hotel_df.loc[:,["City","Country","Lat","Lng"]]

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""


In [78]:
#Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [79]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")
        #delay the code by one second to avoid an error
       
    print("--------------------Ending Search")

Retrieving Results for Index 0: torbay.
Closest hotel in torbay is Flamingo Villas Boutique Hotel.
--------------------Ending Search
Retrieving Results for Index 1: raudeberg.
Closest hotel in raudeberg is Isa Hotel.
--------------------Ending Search
Retrieving Results for Index 2: vad.
Closest hotel in vad is Sarawat Park Hotel & Resort.
--------------------Ending Search
Retrieving Results for Index 3: popondetta.
Closest hotel in popondetta is Altay Water Conservancy Hotel.
--------------------Ending Search
Retrieving Results for Index 4: irbil.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 5: pitrufquen.
Closest hotel in pitrufquen is Bumbarash.
--------------------Ending Search
Retrieving Results for Index 6: axim.
Closest hotel in axim is Desert Gat.
--------------------Ending Search
Retrieving Results for Index 7: pierre.
Closest hotel in pierre is Danesh Hotel.
--------------------Ending Search


In [81]:
#Store into variable named hotel_df
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Lat,Lng,Hotel Name
1,raudeberg,AU,-20.7333,139.5000,Isa Hotel
2,vad,SA,18.2164,42.5053,Sarawat Park Hotel & Resort
3,popondetta,CN,47.8667,88.1167,Altay Water Conservancy Hotel
4,irbil,KZ,52.4994,73.0975,
5,pitrufquen,RU,52.3710,84.3873,Bumbarash
6,axim,IL,31.6100,34.7642,Desert Gat
7,pierre,IR,36.1683,54.3480,Danesh Hotel


In [73]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
locations

,Lat,Lng
0,-22.9575,14.5053
1,-20.7333,139.5000
2,18.2164,42.5053
3,47.8667,88.1167
4,52.4994,73.0975
5,52.3710,84.3873
6,31.6100,34.7642
7,36.1683,54.3480


In [74]:
#Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use the gmaps.figure passing a zoom_level of 2 and a center point so the map displays properly at a readable size
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
markers = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…